In [1]:
#standard ds imports
import pandas as pd
import numpy as np
#viz and stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# .py imports
#import wranglerer as wr
#import modeling as md
import os
#sklearn imports
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#CATboost imports
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import explore_r as ex
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('prepped_data_stad.csv')

In [3]:
df = df.drop(columns=['date','home_score',
       'home_wins', 'away_score', 'away_wins','total_scores'])
df['spread'] = abs(df['spread'])

In [4]:
X_train, y_train, X_validate, y_validate, X_test, y_test = ex.train_validate_test(df,'is_under')

In [5]:
X_train.shape, y_train.shape, X_validate.shape, y_validate.shape, X_test.shape, y_test.shape 

((6471, 14), (6471,), (2394, 14), (2394,), (1946, 14), (1946,))

In [6]:
X_train.head()

,day_of_week,start_time,week_num,stadium,temp,humidity,wind,spread,ou,abnormal_start,is_playoff,playoff_implications,is_turf,is_outdoor
1713,Sunday,1,12,Georgia Dome,72,0,0,4.0,49.0,0,0,1,1,0
2164,Sunday,1,17,NRG Stadium,72,0,0,7.5,38.0,0,0,1,0,0
2554,Sunday,8,9,Reliant Stadium,72,0,0,1.0,42.5,1,0,0,0,0
3117,Sunday,1,6,Heinz Field,58,52,10,12.5,40.0,0,0,0,0,1
9819,Sunday,1,13,Shea Stadium,51,49,14,5.0,39.5,0,0,1,0,1


# MODELING
## CATBoost

In [8]:
# Create and fit the thing
CATb = CatBoostClassifier(verbose=False,depth=5)
CATb.fit(X_train,y_train,cat_features=['day_of_week','start_time','stadium'])
CATb_preds = CATb.predict(X_train)
pd.crosstab(CATb_preds,y_train) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

is_under,0,1
row_0,,
0,2111,1137
1,1069,2154


In [9]:
print(f'Accuracy-Train {round(CATb.score(X_train,y_train),4)}')
print(f'Accuracy-Validate {round(CATb.score(X_validate,y_validate),4)}')
print(classification_report(y_train,CATb_preds))
print(classification_report(y_validate,CATb.predict(X_validate)))

Accuracy-Train 0.6591
Accuracy-Validate 0.5063
              precision    recall  f1-score   support

           0       0.65      0.66      0.66      3180
           1       0.67      0.65      0.66      3291

    accuracy                           0.66      6471
   macro avg       0.66      0.66      0.66      6471
weighted avg       0.66      0.66      0.66      6471

              precision    recall  f1-score   support

           0       0.50      0.51      0.50      1178
           1       0.51      0.51      0.51      1216

    accuracy                           0.51      2394
   macro avg       0.51      0.51      0.51      2394
weighted avg       0.51      0.51      0.51      2394



## CATboost grid_search CROSS_VALIDATION k=5

In [13]:
param_grid = {
    'verbose': [False],
    'depth': [5, 10,15]
}
gr_search = GridSearchCV(CatBoostClassifier(),
                      param_grid)

In [14]:
gr_search

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x1293d3af0>,
             param_grid={'depth': [5, 10, 15], 'verbose': [False]})

In [15]:
gr_search.fit(X_train, y_train,cat_features=['day_of_week','start_time','stadium'])

GridSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x1293d3af0>,
             param_grid={'depth': [5, 10, 15], 'verbose': [False]})

In [16]:
results = gr_search.cv_results_
results_df_init = pd.DataFrame(results)
results_df_init.shape

(3, 15)

In [17]:
params = pd.DataFrame(results['params'])
results_df_init.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_depth,param_verbose,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.783740,0.030590,0.002472,0.000162,5,False,"{'depth': 5, 'verbose': False}",0.528185,0.512365,0.513910,0.515456,0.525502,0.519084,0.006467,1
1,5.485602,0.029377,0.003909,0.000412,10,False,"{'depth': 10, 'verbose': False}",0.498842,0.503864,0.518547,0.507728,0.513138,0.508424,0.006896,2
2,74.159291,0.671937,0.009315,0.002483,15,False,"{'depth': 15, 'verbose': False}",0.495753,0.513138,0.505410,0.512365,0.510046,0.507342,0.006390,3
